## Paper LBO Model Example

Case Study Source: <http://www.streetofwalls.com/finance-training-courses/private-equity-training/paper-lbo-model-example/>


In [485]:
# Install dependencies
%pip install -r requirements.txt

  Using cached pyarrow-13.0.0.tar.gz (1.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/e5/ee/c5bc0d4b66a9c38165adf86e8b57be6f76868edf5ea23b3bbee3680e7edf/scipy-1.11.3-cp312-cp312-macosx_10_9_x86_64.whl.metadata
  Using cached scipy-1.11.3-cp312-cp312-macosx_10_9_x86_64.whl.metadata (60 kB)
Using cached scipy-1.11.3-cp312-cp312-macosx_10_9_x86_64.whl (37.1 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [287 lines of output]
      <string>:34: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      WARNING setuptools_scm.pyproject_reading toml section missing 'pyproject.toml does not contain a tool.setuptools_scm section'
      running bdist_whee

In [487]:
import polars as pl
import numpy_financial as npf
import numpy as np
from dataclasses import dataclass

### 1. Entry Assumptions

Encapsulating assumptions as classes for maintainability.

As a monthly statement I'll just assume year 1's monthly revenue is 1/12 of the annual revenue.

Also the dataclasses are immutable to prevent accidental changes.


In [576]:
# Entry price assumptions
@dataclass(frozen=True)
class Entry:
    entry_multiple: float = 5.0
    y1_rev: float = 100.0
    ebitda_margin: float = 0.4
    entry_cost = entry_multiple * y1_rev * ebitda_margin
    y1_rev_monthly = y1_rev / 12


# Debt & Equity Assumptions
@dataclass(frozen=True)
class Debt:
    debt_ratio: float = 0.6
    equity_ratio = 1 - debt_ratio
    debt = debt_ratio * Entry.entry_cost
    equity = equity_ratio * Entry.entry_cost


# Income Statement Assumptions
@dataclass(frozen=True)
class Parameters:
    rev_growth: float = 1.1
    int_rate: float = 0.10
    tax_rate: float = 0.4
    capex_rate: float = 0.15
    working_capital: float = 5 / 12
    dep_n_amor: float = 20 / 12
    project_years: int or float = 6
    total_months = project_years * 12

Assumptions variable instances:


In [577]:
entry = Entry()
params = Parameters()

debt = Debt().debt
debt_ratio = Debt().debt_ratio
equity = Debt().equity
equity_ratio = Debt().equity_ratio
y1_revenue = entry.y1_rev
entry_multiple = entry.entry_multiple
entry_cost = entry.entry_cost
first_month_rev = entry.y1_rev_monthly
rev_growth = params.rev_growth
ebitda_margin = entry.ebitda_margin
int_rate = params.int_rate
tax_rate = params.tax_rate
capex_rate = params.capex_rate
working_capital = params.working_capital
dep_n_amor = params.dep_n_amor
total_months = params.total_months

entry_assumptions = {
    "Entry Multiple": [entry_multiple],
    "EBITDA (Year 1)": [y1_revenue * ebitda_margin],
    "Cost of Acquisition": [entry_cost],
    "Interest Rate": [int_rate],
    "Debt Ratio": [debt_ratio],
    "Equity Ratio": [equity_ratio],
    "Debt": [debt],
    "Equity": [equity],
}
entry_assumptions = pl.DataFrame(data=entry_assumptions)
entry_assumptions

Entry Multiple,EBITDA (Year 1),Cost of Acquisition,Interest Rate,Debt Ratio,Equity Ratio,Debt,Equity
f64,f64,f64,f64,f64,f64,f64,f64
5,40,200,0.1,0.6,0.4,120,80


### 2. Income Statement


Build Income Statement Class:


In [578]:
class IncomeStatement:
    def __init__(
        self,
        year1_rev_monthly: float,
        rev_growth: float,
        ebitda_margin: float,
        int_rate: float,
        tax_rate: float,
        da: float,
        debt: float,
        total_months: int,
    ) -> None:
        self.year1_rev_monthly = year1_rev_monthly
        self.rev_growth = rev_growth
        self.ebitda_margin = ebitda_margin
        self.int_rate = int_rate
        self.tax_rate = tax_rate
        self.da = da
        self.debt = debt
        self.total_months = total_months

    def months(self) -> pl.DataFrame:
        return pl.DataFrame(
            data={
                "Month": [f"Month {month}" for month in range(1, self.total_months + 1)]
            }
        )

    def revenue(self) -> pl.DataFrame:
        revenue = []
        revenue_dict = {"Revenue": revenue}
        month = 0
        while month < self.total_months:
            if len(revenue) == 0:
                revenue.append(self.year1_rev_monthly)
                month += 1
            elif len(revenue) % 12 == 0:
                revenue.append(revenue[-1] * 1.1)
                month += 1
            else:
                revenue.append(revenue[-1])
                month += 1

        revenue = pl.DataFrame(data=revenue_dict)
        return self.months().with_columns(revenue)

    def ebitda(self) -> pl.DataFrame:
        return self.revenue().with_columns(
            (pl.col("Revenue") * self.ebitda_margin).alias("EBITDA")
        )

    # Assuming amortization is constant for now?
    def ebit(self) -> pl.DataFrame:
        statement = self.ebitda().with_columns(
            pl.lit(self.da).alias("Depreciation & Amortization")
        )
        return statement.with_columns(
            (pl.col("EBITDA") - pl.col("Depreciation & Amortization")).alias("EBIT")
        )

    def int_exp(self) -> pl.DataFrame:
        int_exp = self.debt * (((1 + self.int_rate) ** (1 / 12)) - 1)
        return self.ebit().with_columns(pl.lit(int_exp).alias("Less: Interest Expense"))

    def ebt(self) -> pl.DataFrame:
        return self.int_exp().with_columns(
            (pl.col("EBIT") - pl.col("Less: Interest Expense")).alias("EBT")
        )

    def tax(self) -> pl.DataFrame:
        return self.ebt().with_columns(
            (pl.col("EBT") * self.tax_rate).alias("Less: Tax Payable")
        )

    def net_income(self) -> pl.DataFrame:
        return self.tax().with_columns(
            (pl.col("EBT") - pl.col("Less: Tax Payable")).alias("Net Income")
        )

Create an income statement instance:


In [579]:
income_statement = IncomeStatement(
    first_month_rev,
    rev_growth,
    ebitda_margin,
    int_rate,
    tax_rate,
    dep_n_amor,
    debt,
    total_months,
)

lbo_is = income_statement.net_income().with_columns(pl.exclude("Month").round(2))

# If you wish to go for the usual spreadsheet-like format:
# lbo = lbo.with_columns(pl.exclude("Month").round(2)).transpose(include_header=True, header_name="Month", column_names="Month")
lbo_is

Month,Revenue,EBITDA,Depreciation & Amortization,EBIT,Less: Interest Expense,EBT,Less: Tax Payable,Net Income
str,f64,f64,f64,f64,f64,f64,f64,f64
"""Month 1""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 2""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 3""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 4""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 5""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 6""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 7""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 8""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42
"""Month 9""",8.33,3.33,1.67,1.66,0.96,0.71,0.28,0.42


### 3. Calculate Cumulative Levered Free Cash Flow


Build a class for Free Cash Flow:


In [580]:
class FCF:
    def __init__(
        self,
        capex_rate: float,
        working_capital: float,
        da: float,
        income_statement: IncomeStatement,
    ) -> None:
        self.income_statement = income_statement
        self.capex_rate = capex_rate
        self.working_capital = working_capital
        self.da = da

    def capex(self) -> pl.DataFrame:
        statement = self.income_statement.net_income()
        return statement.with_columns(
            (pl.col("Revenue") * self.capex_rate).alias("Less: Capex")
        )

    def work_cap(self) -> pl.DataFrame:
        statement = self.capex()
        return statement.with_columns(
            (pl.lit(self.working_capital).alias("Less: Working Capital"))
        )

    def da_addback(self) -> pl.DataFrame:
        statement = self.work_cap()
        return statement.with_columns(
            (pl.lit(self.da).alias("Add: Depreciation & Amortization"))
        )

    def fcf(self) -> pl.DataFrame:
        statement = self.da_addback()
        return statement.with_columns(
            (
                pl.col("Net Income")
                + pl.col("Add: Depreciation & Amortization")
                - pl.col("Less: Capex")
                - pl.col("Less: Working Capital")
            ).alias("Monthly FCF")
        )

    def fcf_only(self) -> pl.DataFrame:
        statement = self.fcf()
        return statement.select(
            pl.col("Month"),
            pl.col("Net Income"),
            pl.col("Less: Capex"),
            pl.col("Less: Working Capital"),
            pl.col("Add: Depreciation & Amortization"),
            pl.col("Monthly FCF"),
        )

    def cfi(self) -> pl.DataFrame:
        statement = self.fcf_only()
        cfi = {"CFI": []}
        for i in range(total_months):
            if i == 0:
                cfi["CFI"].append(-equity)
            else:
                cfi["CFI"].append(0)
        cfi = pl.DataFrame(data=cfi)
        return statement.with_columns(cfi)

    def cum_fcf(self) -> pl.DataFrame:
        statement = self.cfi()
        return statement.with_columns(
            pl.col("Monthly FCF").cumsum().alias("Cumulative FCF") + pl.col("CFI")
        )

### 4. Exit Value & Returns


In [582]:
@dataclass
class Exit:
    def __init__(
        self,
        exit_multiple: float,
        lbo_fcf: pl.DataFrame,
        lbo_is: pl.DataFrame,
        equity: float,
        debt: float,
    ) -> None:
        self.exit_multiple = exit_multiple
        self.lbo_fcf = lbo_fcf
        self.lbo_is = lbo_is
        self.equity = equity
        self.debt = debt

    def exit_table(self) -> pl.DataFrame:
        table = (
            self.lbo_is.select(pl.col("EBITDA"))
            .tail(12)
            .sum()
            .rename({"EBITDA": "EBITDA (LTM)"})
        )
        table = table.with_columns(pl.lit(self.exit_multiple).alias("Exit Multiple"))
        table = table.with_columns(pl.lit(self.debt).alias("Beginning Debt"))

        return table

    def total_enterprise_val(self) -> pl.DataFrame:
        df = self.exit_table().with_columns(
            (pl.col("EBITDA (LTM)") * self.exit_multiple).alias(
                "Total Enterprise Value"
            )
        )
        return df

    def exit_debt(self) -> pl.DataFrame:
        fcf = self.lbo_fcf.filter(pl.col("Month") == "Month 60").select(
            "Cumulative FCF"
        )
        fcf = fcf.with_columns(
            (self.debt - pl.col("Cumulative FCF")).alias("Remaining Debt")
        )
        return self.total_enterprise_val().with_columns(fcf)

    def equity_val(self) -> pl.DataFrame:
        table = self.exit_debt()
        table = table.with_columns(
            (pl.col("Total Enterprise Value") - pl.col("Remaining Debt")).alias(
                "Equity Value"
            )
        )
        return table

    def exit_no_irr(self) -> pl.DataFrame:
        table = self.equity_val()
        table = table.with_columns(
            (pl.col("Equity Value") / self.equity).alias("Exit Multiple")
        )
        return table

Create a FCF & exit table instance,
alter the output of irr() to display either the FCF or exit values.


In [584]:
lbo_fcf = FCF(
    capex_rate,
    working_capital,
    dep_n_amor,
    income_statement=income_statement,
).cum_fcf()

exit_table = Exit(
    exit_multiple=5,
    lbo_fcf=lbo_fcf,
    lbo_is=lbo_is,
    equity=equity,
    debt=debt,
).exit_no_irr()


def irr(exit_table: pl.DataFrame, lbo_fcf: pl.DataFrame, output: str) -> pl.DataFrame:
    table = exit_table

    exit_earning = {"Exit Earnings": []}

    for i in range(total_months):
        if len(exit_earning["Exit Earnings"]) + 1 < total_months - 12:
            exit_earning["Exit Earnings"].append(0)
        elif len(exit_earning["Exit Earnings"]) + 1 == total_months - 12:
            exit_earning["Exit Earnings"].append(table["Equity Value"][0])
        elif len(exit_earning["Exit Earnings"]) + 1 <= total_months:
            exit_earning["Exit Earnings"].append(0)

    exit_earning = pl.DataFrame(data=exit_earning)
    lbo_fcf = lbo_fcf.with_columns(exit_earning)
    lbo_fcf = lbo_fcf.with_columns(
        (pl.col("Cumulative FCF") + pl.col("Exit Earnings")).alias("Total CF")
    )

    series = lbo_fcf.get_column("Total CF")
    irr = npf.irr(series)
    exit_table = table.with_columns(pl.lit(irr).alias("IRR"))

    if output == "exit":
        return exit_table
    else:
        return lbo_is


# Change output to "exit" to get exit table, otherwise it will return the FCF.
irr(exit_table, lbo_fcf, output="exit")

EBITDA (LTM),Exit Multiple,Beginning Debt,Total Enterprise Value,Cumulative FCF,Remaining Debt,Equity Value,IRR
f64,f64,f64,f64,f64,f64,f64,f64
64.43999999999998,2.9697201668391964,120,322.19999999999993,35.37761334713579,84.62238665286421,237.57761334713572,0.0831538038898052


Polars does not have a rounding function to dataframes yet,

apologies for the excessive decimal points. lol

Also the original case study's cashflow doesn't seem to incorporate the initial acquisition cost and the desposition value,
therefore the IRR is different.
